In [ ]:
!pip install opencv-python
!pip install scikit-image

from skimage import color, feature
from skimage.feature import hog
from skimage.io import imread
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib import image as mpimg
import os
import glob
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [ ]:
img=np.array(mpimg.imread('.gitignore/dataset/train/freshapples/rotated_by_15_Screen Shot 2018-06-08 at 4.59.36 PM.png'))
img.setflags(write=1)
print('Image : ',img.shape)
plt.imshow(img)
plt.show()

In [ ]:
import cv2

# Redimensionar la imatge
resized_img = cv2.resize(img, (150, 150))

# Mostrar la imatge redimensionada
plt.imshow(resized_img)
plt.show()
print(resized_img.shape)

Nou tamany de la imatge per a poder ser procesada pel model d'aprenentatge més fàcilment. 

A continuació convertirem la imatge a RGB seleccionant només aquests canals, convertirem les imatges a una escala de grisos eliminant la complexitat dels colors i ens centrarem en l'estructura per a detectar formes i textures de les imatges mitjançant la tècnica HOG. 

In [ ]:
from skimage import color, feature
# Remove the alpha channel from the image
rgb_img = resized_img[:, :, :3]


# Convert the image to grayscale
gray_img = color.rgb2gray(rgb_img)

# Compute HOG features for the grayscale image
fd, hog_image = feature.hog(gray_img, orientations=9, pixels_per_cell=(8, 8),
                            cells_per_block=(2, 2), visualize=True)

print(fd.shape)
print(fd)
print(hog_image.shape)
# Display the HOG image
plt.axis("off")
plt.imshow(hog_image, cmap="gray")

**Procesament de imatges Fresh Apples:**

A través del Histogram of Oriented Objects podem extreure les característiques de les Fresh Apples

Llegim tota l'iformació de les dades del train y les guardem junt amb les labels

In [ ]:
# Inicializar listas globales para características y etiquetas
data = []
labels = []


# Definir rutas y etiquetas correspondientes
freshfruits = {
    "freshapples": "freshapple",
    "freshbanana": "freshbanana",
    "freshoranges": "freshorange"
}

rottenfruits = {
    "rottenapples": "rottenapples",
    "rottenbanana": "rottenbanana",
    "rottenoranges": "rottenoranges"
}

# Procesar cada tipo de fruta frescas
for fruit_folder, label in freshfruits.items():
    folder_path = f".gitignore/dataset/train/{fruit_folder}/*.png"
    for entry in glob.glob(folder_path):
        img = cv2.imread(entry)  # Leer imagen
        resized_img = cv2.resize(img, (64, 32))  # Redimensionar imagen
        gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)  # Convertir a escala de grises
        fd = hog(gray_img)  # Calcular características HOG
        data.append(fd)  # Agregar características
        labels.append(label)  # Agregar etiqueta correspondiente
    print(f"{label.capitalize()} Images: {len(glob.glob(folder_path))}")


# Procesar cada tipo de fruta no frescas
for fruit_folder, label in rottenfruits.items():
    folder_path = f".gitignore/dataset/train/{fruit_folder}/*.png"
    for entry in glob.glob(folder_path):
        img = cv2.imread(entry)  # Leer imagen
        resized_img = cv2.resize(img, (64, 32))  # Redimensionar imagen
        gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)  # Convertir a escala de grises
        fd = hog(gray_img)  # Calcular características HOG
        data.append(fd)  # Agregar características
        labels.append(label)  # Agregar etiqueta correspondiente
    print(f"{label.capitalize()} Images: {len(glob.glob(folder_path))}")


# Verificar la longitud de los datos y las etiquetas
print("Total number of images: " + str(len(data)))
print("Total number of labels: " + str(len(labels)))


Entrenem una Maquina de Vectors de Suport amb estrategia One Vs One 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Convertir los datos y etiquetas a arrays
data = np.array(data)
labels = np.array(labels)

# Normalizar las etiquetas para que sean solo tipos de frutas (ignorando si están frescas o podridas)
def normalize_label(label):
    if "apple" in label:
        return "apple"
    elif "banana" in label:
        return "banana"
    elif "orange" in label:
        return "orange"

normalized_labels = np.array([normalize_label(label) for label in labels])

# Codificar etiquetas como valores numéricos
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(normalized_labels)

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.3, random_state=42, stratify=encoded_labels)

# Configurar el modelo con la estrategia One-vs-One
ovo_classifier = OneVsOneClassifier(SVC(kernel='linear', probability=True, random_state=42))

# Entrenar el modelo
ovo_classifier.fit(X_train, y_train)

# Evaluar el modelo
y_pred = ovo_classifier.predict(X_test)

# Mostrar resultados
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=encoder.classes_))
